# Memantau Data Drift

Seiring waktu, model dapat menjadi kurang efektif dalam memprediksi secara akurat karena perubahan tren dalam data fitur. Fenomena ini dikenal sebagai *data drift*, dan penting untuk memantau solusi pembelajaran mesin Anda untuk mendeteksinya sehingga Anda dapat melatih kembali model Anda jika perlu.

Di lab ini, Anda akan mengonfigurasi pemantauan data drift untuk himpunan data.

## Sebelum memulai

Selain versi terbaru paket **azureml-sdk** dan **azureml-widgets**, Anda memerlukan paket **azureml-datadrift** untuk menjalankan kode di notebook ini. Jalankan sel di bawah ini untuk memverifikasi bahwa itu diinstal.

In [ ]:
pip show azureml-datadrift

## Menghubungkan ke ruang kerja Anda

Dengan menginstal paket SDK yang diperlukan, sekarang Anda siap untuk terhubung ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## Membuat himpunan data *garis besar*

Untuk memantau himpunan data untuk penyimpangan data, Anda harus mendaftarkan himpunan data *garis besar* (biasanya himpunan data yang digunakan untuk melatih model Anda) untuk digunakan sebagai titik perbandingan dengan data yang dikumpulkan di masa mendatang. 

In [ ]:
from azureml.core import Datastore, Dataset
from azureml.data.datapath import DataPath

# Upload the baseline data
default_ds = ws.get_default_datastore()
Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'diabetes-data/')
                              )

# Create and register the baseline dataset
print('Registering baseline dataset...')
baseline_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-baseline/*.csv'))
baseline_data_set = baseline_data_set.register(workspace=ws, 
                           name='diabetes baseline',
                           description='diabetes baseline data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Baseline dataset registered!')

## Membuat himpunan data *target*

Seiring waktu, Anda dapat mengumpulkan data baru dengan fitur yang sama seperti data pelatihan garis besar Anda. Untuk membandingkan data baru ini dengan garis besar data, Anda harus menentukan himpunan data target yang menyertakan fitur yang ingin Anda analisis untuk penyimpangan data serta bidang stempel waktu yang menunjukkan titik waktu saat data baru saat ini, hal ini memungkinkan Anda untuk mengukur penyimpangan data selama interval temporal. Stempel waktu dapat berupa bidang dalam himpunan data itu sendiri, atau berasal dari folder dan pola nama file yang digunakan untuk menyimpan data. Misalnya, Anda mungkin menyimpan data baru dalam hierarki folder yang terdiri dari folder untuk tahun tersebut, yang berisi folder untuk bulan tersebut, yang selanjutnya berisi folder untuk hari itu; atau Anda bisa mengodekan tahun, bulan, dan hari dalam nama file seperti ini: *data_2020-01-29.csv*; yang merupakan pendekatan yang diambil dalam kode berikut:

In [ ]:
import datetime as dt
import pandas as pd

print('Generating simulated data...')

# Load the smaller of the two data files
data = pd.read_csv('data/diabetes2.csv')

# We'll generate data for the past 6 weeks
weeknos = reversed(range(6))

file_paths = []
for weekno in weeknos:
    
    # Get the date X weeks ago
    data_date = dt.date.today() - dt.timedelta(weeks=weekno)
    
    # Modify data to ceate some drift
    data['Pregnancies'] = data['Pregnancies'] + 1
    data['Age'] = round(data['Age'] * 1.2).astype(int)
    data['BMI'] = data['BMI'] * 1.1
    
    # Save the file with the date encoded in the filename
    file_path = 'data/diabetes_{}.csv'.format(data_date.strftime("%Y-%m-%d"))
    data.to_csv(file_path)
    file_paths.append(file_path)

# Upload the files
path_on_datastore = 'diabetes-target'
default_ds.upload_files(files=file_paths,
                       target_path=path_on_datastore,
                       overwrite=True,
                       show_progress=True)

# Use the folder partition format to define a dataset with a 'date' timestamp column
partition_format = path_on_datastore + '/diabetes_{date:yyyy-MM-dd}.csv'
target_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, path_on_datastore + '/*.csv'),
                                                       partition_format=partition_format)

# Register the target dataset
print('Registering target dataset...')
target_data_set = target_data_set.with_timestamp_columns('date').register(workspace=ws,
                                                                          name='diabetes target',
                                                                          description='diabetes target data',
                                                                          tags = {'format':'CSV'},
                                                                          create_new_version=True)

print('Target dataset registered!')

## Membuat pemantauan drift data

Sekarang Anda siap membuat pemantauan drift data untuk data diabetes. Pemantauan drift data akan berjalan secara berkala atau sesuai permintaan untuk membandingkan garis besar himpunan data dengan himpunan data target, yang akan ditambahkan data baru seiring waktu.

### Membuat target komputasi

Untuk mengeksekusi pemantauan drift data, Anda memerlukan target komputasi. Jalankan sel berikut untuk menentukan kluster komputasi (jika tidak ada, kluster akan dibuat).

> **Penting**: Ubah *your-compute-cluster* menjadi nama kluster komputasi Anda dalam kode di bawah ini sebelum menjalankannya! Nama kluster harus berupa nama unik global dengan panjang antara 2 hingga 16 karakter. Karakter yang valid adalah huruf, angka, dan - karakter.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **Catatan**: Instans dan kluster komputasi didasarkan pada gambar mesin virtual Azure standar. Untuk latihan ini, gambar *Standard_DS11_v2* direkomendasikan untuk mencapai keseimbangan optimal antara biaya dan performa. Jika langganan Anda memiliki kuota yang tidak menyertakan gambar ini, pilih gambar alternatif; tetapi perlu diingat bahwa gambar yang lebih besar mungkin memerlukan biaya lebih tinggi dan gambar yang lebih kecil mungkin tidak cukup untuk menyelesaikan tugas. Atau, minta administrator Azure Anda untuk memperbesar kuota Anda.

### Menentukan pemantauan drift data

Sekarang Anda siap menggunakan kelas **DataDriftDetector** untuk menentukan monitor penyimpangan data untuk data Anda. Anda dapat menentukan fitur yang ingin Anda pantau untuk penyimpangan data, nama target komputasi yang akan digunakan untuk menjalankan proses pemantauan, frekuensi perbandingan data, ambang penyimpangan data di mana pemberitahuan harus dipicu, dan latensi (dalam jam) untuk memungkinkan pengumpulan data.

In [ ]:
from azureml.datadrift import DataDriftDetector

# set up feature list
features = ['Pregnancies', 'Age', 'BMI']

# set up data drift detector
monitor = DataDriftDetector.create_from_datasets(ws, 'mslearn-diabates-drift', baseline_data_set, target_data_set,
                                                      compute_target=cluster_name, 
                                                      frequency='Week', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=24)
monitor

## Mengisi ulang pemantauan drift data

Anda memiliki himpunan data garis besar dan himpunan data target yang mencakup pengumpulan data mingguan yang disimulasikan selama enam minggu. Anda dapat menggunakan ini untuk mengisi ulang pemantauan sehingga dapat menganalisis penyimpangan data antara garis besar asli dan data target.

> **Catatan** Ini mungkin membutuhkan waktu untuk dijalankan, karena target komputasi harus dimulai untuk menjalankan analisis backfill. Widget mungkin tidak selalu diperbarui untuk menampilkan status, jadi klik tautan untuk mengamati status eksperimen di studio Azure Machine Learning!

In [ ]:
from azureml.widgets import RunDetails

backfill = monitor.backfill(dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())

RunDetails(backfill).show()
backfill.wait_for_completion()

## Menganalisis drift data

Anda dapat menggunakan kode berikut untuk memeriksa drift data untuk titik waktu yang dikumpulkan dalam proses eksekusi backfill.

In [ ]:
drift_metrics = backfill.get_metrics()
for metric in drift_metrics:
    print(metric, drift_metrics[metric])

Anda juga dapat memvisualisasikan metrik penyimpangan data di [Azure Machine Learning studio](https://ml.azure.com) dengan mengikuti langkah-langkah berikut:

1. Pada halaman **Data**, lihat tab **Pemantauan himpunan data**.
2. Klik pemantauan drift data yang ingin Anda lihat.
3. Pilih rentang tanggal yang Anda ingin lihat metrik drift datanya (jika diagram kolom tidak menampilkan data beberapa minggu, tunggu sekitar satu menit dan klik **Refresh**).
4. Periksa bagan di bagian **Ringkasan drift** di bagian atas, yang menunjukkan besaran drift keseluruhan dan kontribusi drift per fitur.
5. Jelajahi bagan di bagian **Detail fitur** di bagian bawah, yang memungkinkan Anda melihat berbagai ukuran drift untuk fitur individual.

> **Catatan**: Untuk membantu memahami metrik drift data, lihat [Cara memantau himpunan data](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets#understanding-data-drift-results) dalam dokumentasi Azure Machine Learning.

## Menjelajahi lebih lanjut

Lab ini dirancang untuk memperkenalkan Anda pada konsep dan prinsip pemantauan drift data. Untuk mempelajari lebih lanjut tentang memantau drift data menggunakan himpunan data, lihat [Mendeteksi drift data pada himpunan data](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets) di dokumentasi Azure machine Learning.

Anda juga dapat mengumpulkan data dari layanan yang diterbitkan dan menggunakannya sebagai himpunan data target untuk pemantauan datadrift. Lihat [Mengumpulkan data dari model dalam produksi](https://docs.microsoft.com/azure/machine-learning/how-to-enable-data-collection) untuk detailnya.
